### Importing the data

#### Initializing With World Development Indicator

In [1]:
import pandas as pd
import numpy as np

world_development_indicator_df = pd.read_csv("../2. Data Extraction\Old Data\Multi-Category Factor Data (Added)\world_bank_group.csv")
income_wages_df = pd.read_csv("..\\2. Data Extraction\Old Data\Economic Factor Data (Added, With Name Mismatches)\income_wages.csv")
# Solution for selecting a specific sheet of an excel file found from the answer of Vaibhav Jadhav at https://stackoverflow.com/questions/71527992/pandas-dataframe-to-specific-sheet-in-a-excel-file-without-losing-formatting
taxes_df = pd.read_excel("..\\2. Data Extraction\Old Data\Economic Factor Data (Added, With Name Mismatches)\world-imf2024.xlsx",sheet_name="Data")
rule_of_law_df = pd.read_csv("..\\2. Data Extraction\Old Data\Political Factor Data\\rule_of_law_index.csv")



country_name = world_development_indicator_df["Country Name"].unique()
gdp_growth = []
pop_density = []
education_years = []
gdp_per_capita = []
electricity_access = []
birth_rate = []
young_pop_percentage_male = []
young_pop_percentage_female = []


# the use of "index" below is from a question on the use of iterrows() in a for loop, answered by waitingkuo https://stackoverflow.com/questions/16476924/how-can-i-iterate-over-rows-in-a-pandas-dataframe
for index, wdi_df_row in world_development_indicator_df.iterrows():
	output_value = ""

	if wdi_df_row["Series Name"] != "Population density (people per sq. km of land area)" and wdi_df_row["2023 [YR2023]"] == ".." : output_value = None
	elif wdi_df_row["Series Name"] == "Population density (people per sq. km of land area)" and wdi_df_row["2022 [YR2022]"] == ".." : output_value = None
	else: 
		if wdi_df_row["Series Name"] != "Population density (people per sq. km of land area)": output_value = wdi_df_row["2023 [YR2023]"]
		else: output_value = wdi_df_row["2022 [YR2022]"]

	if wdi_df_row["Series Name"] == "GDP per capita growth (annual %)": gdp_growth.append(output_value)
	elif wdi_df_row["Series Name"] == "Population density (people per sq. km of land area)": pop_density.append(output_value) # Best set as 2022, since there is no data for any country in 2023
	elif wdi_df_row["Series Name"] == "Compulsory education, duration (years)": education_years.append(output_value)
	elif wdi_df_row["Series Name"] == "GDP per capita (current US$)": gdp_per_capita.append(output_value)
	elif wdi_df_row["Series Name"] == "Access to electricity (% of population)": electricity_access.append(output_value)
	elif wdi_df_row["Series Name"] == "Birth rate, crude (per 1,000 people)": birth_rate.append(output_value)
	elif wdi_df_row["Series Name"] == "Population ages 20-24, male (% of male population)": young_pop_percentage_male.append(output_value)
	elif wdi_df_row["Series Name"] == "Population ages 20-24, female (% of female population)": young_pop_percentage_female.append(output_value)


# Figured out how to make an empty list with help from jL4's answer at https://stackoverflow.com/questions/43336837/making-equal-size-lists-in-python
avg_hourly_earnings = []
avg_hourly_earnings.extend([None] * len(country_name))

taxes_percentage = []
taxes_percentage.extend([None] * len(country_name))

fundamental_rights = []
fundamental_rights.extend([None] * len(country_name))

legal_system_fairness = []
legal_system_fairness.extend([None] * len(country_name))

df = pd.DataFrame({
	"Country Name":country_name,
	"GDP per capita growth (annual %)":gdp_growth,
	"Population density (people per sq. km of land area)":pop_density,
	"Compulsory education, duration (years)":education_years,
	"GDP per capita (current US$)":gdp_per_capita,
	"Access to electricity (% of population)":electricity_access,
	"Birth rate, crude (per 1,000 people)":birth_rate,
	"Population ages 20-24, male (% of male population)":young_pop_percentage_male,
	"Population ages 20-24, female (% of female population)":young_pop_percentage_female,
	"Average hourly earnings of employees":avg_hourly_earnings,
	"Tax revenue, percent of GDP":taxes_percentage,
	"Fundamental Rights":fundamental_rights,
	"Legal System Fairness":legal_system_fairness
	})

#### Adding Average Hourly Earnings

In [2]:
for index, iw_df_row in income_wages_df.iterrows():
	name_mismatched = ""

	# Solution for checking existence of matching country names in the main dataframe found from Akram's answer at https://stackoverflow.com/questions/21319929/how-to-determine-whether-a-pandas-column-contains-a-particular-value
	if iw_df_row["ref_area.label"] in df["Country Name"].values:
		for index, df_row in df.iterrows():
			if df_row["Country Name"] == iw_df_row["ref_area.label"]:
				df_row["Average hourly earnings of employees"] = iw_df_row["obs_value"]
				break
	# hardcoding was required for these countries. Due to the mismatch of country name, they triggered the else block below
	elif iw_df_row["ref_area.label"] == "Egypt": name_mismatched = "Egypt, Arab Rep."
	elif iw_df_row["ref_area.label"] == "United Kingdom of Great Britain and Northern Ireland": name_mismatched = "United Kingdom"
	elif iw_df_row["ref_area.label"] == "Republic of Korea": name_mismatched = "Korea, Rep." # determined to be south korea https://history.state.gov/countries/korea-south#:~:text=Republic%20of%20Korea%20(South%20Korea,Countries%20%2D%20Office%20of%20the%20Historian
	elif iw_df_row["ref_area.label"] == "Republic of Moldova": name_mismatched = "Moldova"
	elif iw_df_row["ref_area.label"] == "Slovakia": name_mismatched = "Slovak Republic"
	elif iw_df_row["ref_area.label"] == "Türkiye": name_mismatched = "Turkiye"
	elif iw_df_row["ref_area.label"] == "Tanzania, United Republic of": name_mismatched = "Tanzania"
	elif iw_df_row["ref_area.label"] == "United States of America": name_mismatched = "United States"
	else:
		print(iw_df_row["ref_area.label"])
		print("I do not exist")
	for index, df_row in df.iterrows():
		if df_row["Country Name"] == name_mismatched:
			df_row["Average hourly earnings of employees"] = iw_df_row["obs_value"]
			# print("Successfully assigned ", df_row["Country Name"]," the value ",iw_df_row["obs_value"])
			break

#### Adding Tax Revenue

In [3]:
count = 0
for index, tax_df_row in taxes_df.iterrows():
	if tax_df_row["year"] != 2019: continue
	elif tax_df_row["property_ShortForm_en_displayNam"] in df["Country Name"].values:
		for index, df_row in df.iterrows():
			if df_row["Country Name"] == tax_df_row["property_ShortForm_en_displayNam"]:
				df_row["Tax revenue, percent of GDP"] = tax_df_row["TaxRev"]
				break
	else:
		print(tax_df_row["property_ShortForm_en_displayNam"])
		print("I do not exist")
		count += 1

print(count)

UAE
I do not exist
The Bahamas
I do not exist
Democratic Republic of the Congo
I do not exist
Republic of Congo
I do not exist
Czech Republic
I do not exist
Egypt
I do not exist
Micronesia
I do not exist
The Gambia
I do not exist
Hong Kong SAR
I do not exist
Iran
I do not exist
Korea
I do not exist
Lao P.D.R
I do not exist
Macao SAR
I do not exist
Russia
I do not exist
Türkiye
I do not exist
Venezuela
I do not exist
Vietnam
I do not exist
Yemen
I do not exist
18


#### Adding Rule Of Law Index

In [4]:
count = 0
for index, rol_df_row in rule_of_law_df.iterrows():
	if rol_df_row["Country"] in df["Country Name"].values:
		for index, df_row in df.iterrows():
			if df_row["Country Name"] == rol_df_row["Country"]:
				df_row["Fundamental Rights"] = rol_df_row["Fundamental Rights"]
				df_row["Legal System Fairness"] = rol_df_row["Civil Justice + Criminal Justice"]
				break
	else:
		print(rol_df_row["Country"])
		print("I do not exist")
		count += 1

print(count)

Hong Kong, China
I do not exist
St. Vincent and G...
I do not exist
The Bahamas
I do not exist
Bosnia and Herz...
I do not exist
Vietnam
I do not exist
The Gambia
I do not exist
Côte d'Ivoire
I do not exist
Türkiye
I do not exist
8


#### Checking Null Values

In [5]:
df.isnull().sum()
# NULL VALUES:
#  Access to electricity (% of population)                    2
#  GDP per capita (current US$)                              22
#  Compulsory education, duration (years)                    20
#  Population density (people per sq. km of land area)        7
#  GDP per capita growth (annual %)                          22
#  Average hourly earnings of employees                     164
#  Tax revenue, percent of GDP                               43

Country Name                                                0
GDP per capita growth (annual %)                           22
Population density (people per sq. km of land area)         7
Compulsory education, duration (years)                     20
GDP per capita (current US$)                               22
Access to electricity (% of population)                     2
Birth rate, crude (per 1,000 people)                        0
Population ages 20-24, male (% of male population)          0
Population ages 20-24, female (% of female population)      0
Average hourly earnings of employees                      164
Tax revenue, percent of GDP                                43
Fundamental Rights                                         83
Legal System Fairness                                      83
dtype: int64